In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
import statistics
import re

import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Load data
dtset = pd.read_csv("../../Hanoi-Housing-Prices-Prediction/Preprocessing/chungcu_clean.csv")
dtset_columns = dtset.columns.tolist()
print("+ Shape: ", dtset.shape) # (Number of Lines, Number of Columns)
print("+ Column: ", dtset_columns) # Name the attribute columns.
dtset.info()

+ Shape:  (14954, 17)
+ Column:  ['Tiêu đề', 'Dự án', 'Khu vực', 'Ngày đăng', 'Ngày hết hạn', 'Loại tin', 'Mã tin', 'Người đăng', 'Thông tin mô tả', 'Diện tích', 'Mức giá', 'Hướng nhà', 'Hướng ban công', 'Số phòng ngủ', 'Số toilet', 'Pháp lý', 'Nội thất']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14954 entries, 0 to 14953
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Tiêu đề          14954 non-null  object 
 1   Dự án            14954 non-null  object 
 2   Khu vực          14954 non-null  object 
 3   Ngày đăng        14954 non-null  object 
 4   Ngày hết hạn     14954 non-null  object 
 5   Loại tin         14954 non-null  object 
 6   Mã tin           14954 non-null  int64  
 7   Người đăng       14954 non-null  object 
 8   Thông tin mô tả  14954 non-null  object 
 9   Diện tích        14954 non-null  float64
 10  Mức giá          14954 non-null  object 
 11  Hướng nhà        7355 non-null  

### Delete attribute columns

In [3]:
# Print the oldest and newest posting dates
print("Oldest posting date:", dtset['Ngày đăng'].min())
print("Newest posting date:", dtset['Ngày đăng'].max())

# Print the oldest and newest expiration dates
print("Oldest expiration date:", dtset['Ngày hết hạn'].min())
print("Newest expiration date:", dtset['Ngày hết hạn'].max())

Oldest posting date: 2024-01-31
Newest posting date: 2024-03-11
Oldest expiration date: 2024-03-01
Newest expiration date: 2024-04-09


Notice that the gap between the posting date and the expiration date is negligible (only about 1 month) => The range of house price fluctuations in that range will not be too large => Skip this section, drop the "Ngày đăng" and "Ngày hết hạn" columns

"Người đăng" feature has no value => drop 

In [4]:
# Print the initial dataset size
print("Initial dataset size:", dtset.shape)

# Drop specified columns
dtset.drop(['Ngày đăng', 'Ngày hết hạn', 'Người đăng'], axis=1, inplace=True)

# Print the dataset size after removing some attribute columns
print("Dataset size after removing some attribute columns:", dtset.shape)

Initial dataset size: (14954, 17)
Dataset size after removing some attribute columns: (14954, 14)


### Processing the "Khu vực" column

In [5]:
dtset['Khu vực'].unique()

array(['đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.',
       'Phường La Khê, Hà Đông, Hà Nội',
       'Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh Xuân, Hà Nội', ...,
       'Đường Nguyễn Khuyến, Phường Phúc La, Hà Đông, Hà Nội.',
       'Đường Giải Phóng, Phường Bạch Mai, Hai Bà Trưng, Hà Nội',
       'Phố Khâm Thiên, Phường Thổ Quan, Đống Đa, Hà Nội'], dtype=object)

In [6]:
# Loop through the dataset and print full address information
for address in dtset['Khu vực']:
    print(address)

đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.
Phường La Khê, Hà Đông, Hà Nội
Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh Xuân, Hà Nội
69B, Đường Thụy Khuê, Phường Thụy Khuê, Tây Hồ, Hà Nội
Đường Phạm Hùng, Phường Mễ Trì, Nam Từ Liêm, Hà Nội
35 Đường Lê Văn Lương, Phường Nhân Chính, Thanh Xuân, Hà Nội
Đường Dương Văn Bé, Phường Vĩnh Tuy, Hai Bà Trưng, Hà Nội
Đường Thành Thái, Phường Dịch Vọng, Cầu Giấy, Hà Nội
Đường Thành Thái, Phường Dịch Vọng, Cầu Giấy, Hà Nội
Đường Giải Phóng, Phường Tương Mai, Hoàng Mai, Hà Nội
Xã Dương Xá, Gia Lâm, Hà Nội
Phường Mỗ Lao, Hà Đông, Hà Nội
609, Đường Trương Định, Phường Giáp Bát, Hoàng Mai, Hà Nội
Đường Lê Văn Lương, Phường Nhân Chính, Thanh Xuân, Hà Nội
Đường Lương Thế Vinh, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.
25, Đường Lê Văn Lương, Phường Nhân Chính, Thanh Xuân, Hà Nội
Bắc Từ Liêm, Hà Nội
Phố Thái Hà, Phường Trung Liệt, Đống Đa, Hà Nội
Đường Lê Đức Thọ, Phường Mỹ Đình 2, Nam Từ Liêm, Hà Nội
Đường Minh Khai, Phường Vĩnh Tuy, Hai Bà Trưng, Hà 

Add "Đường/Phố" column

In [7]:
# Create a list to store street information
streets = [np.nan for i in range(dtset.shape[0])]
print("Streets length before =", len(streets))  # "streets" is a new column named "Đường" extracted from the "Khu vực" column.
print("Streets sample before:", streets[0:3])
street_exception_list = []  # Contains exceptional cases in a list variable.

Streets length before = 14954
Streets sample before: [nan, nan, nan]


In [8]:
# Create a list to store street information
streets = []

# Loop through the dataset
for index in range(dtset.shape[0]):
    # Skip if the value of 'Khu vực' at index 'index' is NaN
    if pd.isnull(dtset['Khu vực'][index]):
        streets.append(np.nan)  # Append NaN if address is NaN
        continue
    
    # Split address by ", " to extract components
    address_components = dtset['Khu vực'][index].split(", ")

    street_found = False
    # Iterate over components to find street
    for component in address_components:
        if 'đường' in component.lower() or 'phố' in component.lower():
            # Remove 'đường' and 'phố' and leading/trailing whitespaces
            street = component.replace('Đường', '').replace('Phố', '').replace('đường', '').replace('phố', '').strip()
            streets.append(street)
            street_found = True
            break
    
    if not street_found:
        # If neither 'đường' nor 'phố' is found in any component, append NaN
        streets.append(np.nan)

In [9]:
print("Streets length after =", len(streets))
print("Streets sample after:", streets[0:3])
print("Total number of exceptional cases =", len(street_exception_list))

Streets length after = 14954
Streets sample after: ['Mễ Trì', nan, 'Hoàng Đạo Thúy']
Total number of exceptional cases = 0


In [10]:
streets

['Mễ Trì',
 nan,
 'Hoàng Đạo Thúy',
 'Thụy Khuê',
 'Phạm Hùng',
 '35  Lê Văn Lương',
 'Dương Văn Bé',
 'Thành Thái',
 'Thành Thái',
 'Giải Phóng',
 nan,
 nan,
 'Trương Định',
 'Lê Văn Lương',
 'Lương Thế Vinh',
 'Lê Văn Lương',
 nan,
 'Thái Hà',
 'Lê Đức Thọ',
 'Minh Khai',
 'Xuân Đỉnh',
 'Cầu Bươu',
 nan,
 'Đông Dư',
 'Linh',
 'Tố Hữu',
 'Lê Trọng Tấn',
 nan,
 'Đại lộ Thăng Long',
 'Minh Khai',
 'Vũ Trọng Phụng',
 nan,
 'Bùi Thiện Ngộ',
 nan,
 nan,
 nan,
 'Nguyễn Xiển',
 'Láng Hạ',
 nan,
 'Đào Văn Tập',
 nan,
 'Hoàng Đạo Thúy',
 'Tố Hữu',
 'Tam Trinh',
 nan,
 nan,
 'Láng Hạ',
 'Hoàng Minh Giám',
 'Kim Giang',
 'Trần Phú',
 'Lê Quang Đạo',
 'Tố Hữu',
 'Xuân La',
 'Tứ Hiệp',
 'Ngụy Như Kon Tum',
 nan,
 'Trần Phú',
 'Trần Bình',
 'Quang Trung',
 'Phùng Hưng',
 'Huỳnh Thúc Kháng',
 'Tứ Hiệp',
 nan,
 nan,
 'Nguyễn Chí Thanh',
 'Trần Kim Xuyến',
 'Thanh Bình',
 'Võ Chí Công',
 'Trần Bình',
 'Tố Hữu',
 'Nguyễn Tuân',
 'Trần Hữu Dực',
 'Minh Khai',
 'Trần Nguyên Đán',
 nan,
 '21',
 'Phạm Hùng

In [11]:
# Add the "Đường" column to the dataset 
dtset.insert(3, "Đường/Phố", streets, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (14954, 15)


,Tiêu đề,Dự án,Khu vực,Đường/Phố,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Hướng nhà,Hướng ban công,Số phòng ngủ,Số toilet,Pháp lý,Nội thất
0,Chính chủ bán căn hoa hậu view thoáng tầng tru...,Dự án Golden Palace,"đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.",Mễ Trì,Tin thường,37587338,Chính chủ bán căn hoa hậu view thoáng tầng tru...,128.0,35.0,Tây - Bắc,Đông - Nam,3.0,2.0,NaN,NaN
1,"Bán căn hộ CT5 Văn Khê, Hà Đông",Dự án CT5 Văn Khê,"Phường La Khê, Hà Đông, Hà Nội",NaN,Tin thường,39014297,"Bán căn hộ CT5A, Văn Khê, Hà Đông.\n- Diện tíc...",82.0,32.3,Tây - Bắc,Đông - Nam,2.0,2.0,Sổ hồng,Đầy đủ
2,Cần tiền bán nhanh căn hộ cao cấp tòa Diamond ...,Dự án Diamond Flower Tower,"Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh X...",Hoàng Đạo Thúy,Tin thường,39135117,- Chính chủ cần tiền đầu tư nhờ bán gấp căn hộ...,115.0,55.0,Nam,NaN,3.0,2.0,Sổ đỏ/ Sổ hồng,Đầy đủ


Add "Phường/Xã" column

In [12]:
ward = [np.nan for i in range(dtset.shape[0])]  # Create a list named "ward" with NaN values, with the length equal to the number of rows in the dataset.
print("Ward length before =", len(ward))  # "ward" is a new column named "Phường" extracted from the "Khu vực" column.
print("Ward sample before:", ward[0:3])
#ward_dictionary = {} # Contains exceptional cases that cannot immediately construct the "Phường" column.
ward_exception_list = []  # Contains exceptional cases in a list variable.

Ward length before = 14954
Ward sample before: [nan, nan, nan]


In [13]:
for index in range(dtset.shape[0]):
    if(type(dtset['Khu vực'][index]) == float):
        # Skip the current iteration if the value of 'Khu vực' at index 'index' is of type float.
        continue
    if 'Phường' not in dtset['Khu vực'][index] and 'Xã' not in dtset['Khu vực'][index] and 'Thị trấn' not in dtset['Khu vực'][index]:
        # If the value of 'Địa chỉ' at index 'index' does NOT contain the strings 'Phường', 'Xã', and 'Thị trấn':
        #ward_dictionary[index] = df['Khu vực'][index] # Note this down in ward_dictionary.
        ward_exception_list.append(index)  # Note the exception in ward_exception_list for deletion later.
        continue  # and do nothing.
    address = dtset['Khu vực'][index].split(", ")  # Example of an address, then:
    # "Đường Tố Hữu, Phường Dương Nội, Quận Hà Đông, Hà Nội" -> ['Đường Tố Hữu', 'Phường Dương Nội', 'Hà Đông', 'Hà Nội']
    for row in address:
        if 'Phường' in row or 'Xã' in row or 'Thị trấn' in row:
            # If the value of 'Địa chỉ' at index 'index' DOES contain the strings 'Phường', 'Xã', or 'Thị trấn':
            # Use the .strip() method to remove leading and trailing whitespaces.
            new_row = row.replace('Phường', '').replace('Xã', '').replace('Thị trấn', '').strip()
            #new_row = row.strip() # Do not remove 'Phường', 'Xã', or 'Thị trấn' if present.
            if new_row:
                ward[index] = new_row  # Save it as the last result to create a new column.
            else:
                ward[index] = np.nan  # If new_row is empty, assign 'NaN'
            continue
    #print(address)

In [14]:
print("Ward length after =", len(ward))
print("Ward sample after:", ward[0:3])

Ward length after = 14954
Ward sample after: ['Mễ Trì', 'La Khê', 'Nhân Chính']


In [15]:
print("Total number of exceptional cases =", len(ward_exception_list))
#print("ward_dictionary length =", len(ward_dictionary))
#display(ward_dictionary)
print("Example ward_exception_list[0:10]:", ward_exception_list[0:10])
print("Example of exceptional value 1 (in the Dataset):", dtset['Khu vực'][ward_exception_list[0]])
print("Example of exceptional value 1 (in the ward variable):", ward[ward_exception_list[0]])

Total number of exceptional cases = 193
Example ward_exception_list[0:10]: [16, 176, 193, 217, 228, 260, 285, 627, 773, 922]
Example of exceptional value 1 (in the Dataset): Bắc Từ Liêm, Hà Nội
Example of exceptional value 1 (in the ward variable): nan


In [16]:
for index in ward_exception_list:
    print(f"Index: {index}, Value in the Dataset: {dtset['Khu vực'][index]}")

Index: 16, Value in the Dataset: Bắc Từ Liêm, Hà Nội
Index: 176, Value in the Dataset: Quận Hai Bà Trưng, Hà Nội
Index: 193, Value in the Dataset: Quận Cầu Giấy, Hà Nội
Index: 217, Value in the Dataset: Quận Đống Đa, Hà Nội
Index: 228, Value in the Dataset: Quận Đống Đa, Hà Nội
Index: 260, Value in the Dataset: Quận Hoàng Mai, Hà Nội
Index: 285, Value in the Dataset: Hoàng Mai, Hà Nội
Index: 627, Value in the Dataset: Đống Đa, Hà Nội
Index: 773, Value in the Dataset: Quận Nam Từ Liêm, Hà Nội
Index: 922, Value in the Dataset: 609 Trương Định, Quận Hoàng Mai, Hà Nội
Index: 926, Value in the Dataset: Gia Lâm, Hà Nội
Index: 998, Value in the Dataset: Quận Hà Đông, Hà Nội
Index: 1086, Value in the Dataset: Quận Cầu Giấy, Hà Nội
Index: 1142, Value in the Dataset: Huyện Hoài Đức, Hà Nội
Index: 1260, Value in the Dataset: Huyện Hoài Đức, Hà Nội
Index: 1348, Value in the Dataset: Ba Đình, Hà Nội
Index: 1366, Value in the Dataset: Quận Cầu Giấy, Hà Nội
Index: 1433, Value in the Dataset: Quận Tây

In [17]:
# Add the "Phường" column to the dataset
dtset.insert(4, "Phường/Xã", ward, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (14954, 16)


,Tiêu đề,Dự án,Khu vực,Đường/Phố,Phường/Xã,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Hướng nhà,Hướng ban công,Số phòng ngủ,Số toilet,Pháp lý,Nội thất
0,Chính chủ bán căn hoa hậu view thoáng tầng tru...,Dự án Golden Palace,"đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.",Mễ Trì,Mễ Trì,Tin thường,37587338,Chính chủ bán căn hoa hậu view thoáng tầng tru...,128.0,35.0,Tây - Bắc,Đông - Nam,3.0,2.0,NaN,NaN
1,"Bán căn hộ CT5 Văn Khê, Hà Đông",Dự án CT5 Văn Khê,"Phường La Khê, Hà Đông, Hà Nội",NaN,La Khê,Tin thường,39014297,"Bán căn hộ CT5A, Văn Khê, Hà Đông.\n- Diện tíc...",82.0,32.3,Tây - Bắc,Đông - Nam,2.0,2.0,Sổ hồng,Đầy đủ
2,Cần tiền bán nhanh căn hộ cao cấp tòa Diamond ...,Dự án Diamond Flower Tower,"Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh X...",Hoàng Đạo Thúy,Nhân Chính,Tin thường,39135117,- Chính chủ cần tiền đầu tư nhờ bán gấp căn hộ...,115.0,55.0,Nam,NaN,3.0,2.0,Sổ đỏ/ Sổ hồng,Đầy đủ


Add Quận/Huyện column

In [18]:
# Create a list named "district" with NaN values, with the length equal to the number of rows in the dataset.
district = [np.nan] * len(dtset)
print("District length before =", len(district))  # "district" is a new column named "Quận" extracted from the "Địa chỉ" column.
print("District sample before:", district[0:3])
dist_exception_list = []  # Contains exceptional cases in a list variable.

District length before = 14954
District sample before: [nan, nan, nan]


In [19]:
# Loop through the dataset
for index in range(len(dtset)):
    # Skip if the value of 'Khu vực' at index 'index' is NaN
    if pd.isnull(dtset['Khu vực'][index]):
        continue
    
    # Split address by ", " to extract components
    address_components = dtset['Khu vực'][index].split(", ")

    district_found = False
    # Iterate over components to find district
    for component in address_components:
        if 'Quận' in component or 'Huyện' in component:
            # Remove 'Quận' or 'Huyện' and leading/trailing whitespaces
            district_name = component.replace('Quận', '').replace('Huyện', '').strip()
            district[index] = district_name
            district_found = True
            break
    
    if not district_found:
        # If neither 'Quận' nor 'Huyện' is found in any component, consider the last component as district
        if len(address_components) >= 2:
            district[index] = address_components[-2].strip()  # Assuming the district is the second last component
        else:
            dist_exception_list.append(index)  # Note the exception in dist_exception_list for deletion later

In [20]:
print("District length after =", len(district))
print("District sample after:", district[0:3])
print("Total number of exceptional cases =", len(dist_exception_list))

District length after = 14954
District sample after: ['Nam Từ Liêm', 'Hà Đông', 'Thanh Xuân']
Total number of exceptional cases = 1


In [21]:
for index in dist_exception_list:
    print(f"Index: {index}, Value in the Dataset: {dtset['Khu vực'][index]}")

Index: 2475, Value in the Dataset: Hoàng Minh Thảo


In [22]:
print("Row at index 2475 before dropping:")
print(dtset.iloc[2475])


Row at index 2475 before dropping:
Tiêu đề            Chính chủ bán CH penthouse 226m2 - 4PN tòa N01...
Dự án                          Chung cư N01T1 Ngoại Giao Đoàn Hà Nội
Khu vực                                              Hoàng Minh Thảo
Đường/Phố                                                        NaN
Phường/Xã                                                        NaN
Loại tin                                                  Tin thường
Mã tin                                                      37684001
Thông tin mô tả    Bán căn hộ penthouse chung cư N01T1 Ngoại Giao...
Diện tích                                                      226.0
Mức giá                                                         61.9
Hướng nhà                                                        Bắc
Hướng ban công                                                   Nam
Số phòng ngủ                                                     4.0
Số toilet                                                        3.0

In [23]:
# Add the "Quận" column to the dataset 
dtset.insert(5, "Quận/Huyện", district, True)
print("Dataset shape:", dtset.shape)
dtset.head(3)

Dataset shape: (14954, 17)


,Tiêu đề,Dự án,Khu vực,Đường/Phố,Phường/Xã,Quận/Huyện,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Hướng nhà,Hướng ban công,Số phòng ngủ,Số toilet,Pháp lý,Nội thất
0,Chính chủ bán căn hoa hậu view thoáng tầng tru...,Dự án Golden Palace,"đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.",Mễ Trì,Mễ Trì,Nam Từ Liêm,Tin thường,37587338,Chính chủ bán căn hoa hậu view thoáng tầng tru...,128.0,35.0,Tây - Bắc,Đông - Nam,3.0,2.0,NaN,NaN
1,"Bán căn hộ CT5 Văn Khê, Hà Đông",Dự án CT5 Văn Khê,"Phường La Khê, Hà Đông, Hà Nội",NaN,La Khê,Hà Đông,Tin thường,39014297,"Bán căn hộ CT5A, Văn Khê, Hà Đông.\n- Diện tíc...",82.0,32.3,Tây - Bắc,Đông - Nam,2.0,2.0,Sổ hồng,Đầy đủ
2,Cần tiền bán nhanh căn hộ cao cấp tòa Diamond ...,Dự án Diamond Flower Tower,"Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh X...",Hoàng Đạo Thúy,Nhân Chính,Thanh Xuân,Tin thường,39135117,- Chính chủ cần tiền đầu tư nhờ bán gấp căn hộ...,115.0,55.0,Nam,NaN,3.0,2.0,Sổ đỏ/ Sổ hồng,Đầy đủ


In [24]:
# Count the number of NaN values in column 'Quận/Huyện'
nan_district_count = dtset['Quận/Huyện'].isnull().sum()

# Count the number of NaN values in column 'Phường/Xã'
nan_ward_count = dtset['Phường/Xã'].isnull().sum()

# Count the number of NaN values in column'Đường/Phố'
nan_street_count = dtset['Đường/Phố'].isnull().sum()

print("Số lượng giá trị NaN trong cột 'Quận/Huyện':", nan_district_count)
print("Số lượng giá trị NaN trong cột 'Phường/Xã':", nan_ward_count)
print("Số lượng giá trị NaN trong cột 'Đường/Phố':", nan_street_count)

Số lượng giá trị NaN trong cột 'Quận/Huyện': 1
Số lượng giá trị NaN trong cột 'Phường/Xã': 193
Số lượng giá trị NaN trong cột 'Đường/Phố': 3402


In [25]:
# Handle error value
dtset['Quận/Huyện'] = dtset['Quận/Huyện'].astype(str)
contains_Giang_Bien = dtset[dtset['Quận/Huyện'].str.contains('Giang Biên')]
print(contains_Giang_Bien)
dtset.loc[12695, 'Phường/Xã'] = 'Giang Biên'
dtset.loc[12695, 'Quận/Huyện'] = 'Long Biên'

                                                 Tiêu đề  \
12695  Bán gấp căn góc 3PN, 2vs diện tích 82m2 chung ...   

                              Dự án                       Khu vực Đường/Phố  \
12695  Dự án Chung cư Ruby City CT2  Phúc Lợi, Giang Biên, Hà Nội       NaN   

      Phường/Xã  Quận/Huyện    Loại tin    Mã tin  \
12695       NaN  Giang Biên  Tin thường  39187084   

                                         Thông tin mô tả  Diện tích Mức giá  \
12695  Bán gấp căn góc 3pn, 2vs diện tích 82m² chung ...       82.0    29.3   

      Hướng nhà Hướng ban công  Số phòng ngủ  Số toilet Pháp lý Nội thất  
12695       NaN            NaN           3.0        2.0  Sổ đỏ.  Đầy đủ.  


### District classification

In [26]:
# Create a new column named "Phân loại quận" with default value NaN
dtset.insert(loc=6, column='Phân loại quận', value=np.nan)

# List of inner districts
inner_districts = ['Ba Đình', 'Hoàn Kiếm', 'Tây Hồ', 'Cầu Giấy', 'Đống Đa', 'Hai Bà Trưng', 'Hoàng Mai', 'Thanh Xuân', 'Long Biên', 'Bắc Từ Liêm', 'Nam Từ Liêm', 'Hà Đông']

# Mark inner districts
dtset.loc[dtset['Quận/Huyện'].isin(inner_districts), 'Phân loại quận'] = 'Nội thành'

# Mark outer districts
dtset.loc[~dtset['Quận/Huyện'].isin(inner_districts), 'Phân loại quận'] = 'Ngoại thành'

# Drop the row with index 2475 (Row had error value of Khu vực)
dtset.drop(index=2475, inplace=True)

dtset.head()

,Tiêu đề,Dự án,Khu vực,Đường/Phố,Phường/Xã,Quận/Huyện,Phân loại quận,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Hướng nhà,Hướng ban công,Số phòng ngủ,Số toilet,Pháp lý,Nội thất
0,Chính chủ bán căn hoa hậu view thoáng tầng tru...,Dự án Golden Palace,"đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.",Mễ Trì,Mễ Trì,Nam Từ Liêm,Nội thành,Tin thường,37587338,Chính chủ bán căn hoa hậu view thoáng tầng tru...,128.0,35.0,Tây - Bắc,Đông - Nam,3.0,2.0,NaN,NaN
1,"Bán căn hộ CT5 Văn Khê, Hà Đông",Dự án CT5 Văn Khê,"Phường La Khê, Hà Đông, Hà Nội",NaN,La Khê,Hà Đông,Nội thành,Tin thường,39014297,"Bán căn hộ CT5A, Văn Khê, Hà Đông.\n- Diện tíc...",82.0,32.3,Tây - Bắc,Đông - Nam,2.0,2.0,Sổ hồng,Đầy đủ
2,Cần tiền bán nhanh căn hộ cao cấp tòa Diamond ...,Dự án Diamond Flower Tower,"Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh X...",Hoàng Đạo Thúy,Nhân Chính,Thanh Xuân,Nội thành,Tin thường,39135117,- Chính chủ cần tiền đầu tư nhờ bán gấp căn hộ...,115.0,55.0,Nam,NaN,3.0,2.0,Sổ đỏ/ Sổ hồng,Đầy đủ
3,Bán căn hộ Duplex Sun Grand City - 69B Thụy Kh...,Sun Grand City,"69B, Đường Thụy Khuê, Phường Thụy Khuê, Tây Hồ...",Thụy Khuê,Thụy Khuê,Tây Hồ,Nội thành,Tin thường,37509440,Cần bán căn hộ Duplex cao cấp - Sun Grand City...,250.0,96.0,NaN,NaN,4.0,3.0,NaN,NaN
4,Bán căn hộ chung cư Vinhomes West Point. 3PN 9...,Dự án Vinhomes West Point,"Đường Phạm Hùng, Phường Mễ Trì, Nam Từ Liêm, H...",Phạm Hùng,Mễ Trì,Nam Từ Liêm,Nội thành,Tin thường,39159735,Bán căn hộ chung cư Vinhomes West Point.\nCăn ...,93.0,69.9,Tây - Bắc,Đông - Nam,3.0,2.0,Sổ đỏ/ Sổ hồng,Cơ bản


### Handle "Dự án" column

In [27]:
dtset['Dự án'][3000:].nunique()

1154

In [28]:
dtset['Dự án'] = dtset['Dự án'].str.replace('Dự án ', '')
for index, row in dtset.iterrows():
    if 'Chung cư mini' in row['Dự án']:
        dtset.at[index, 'Loại hình chung cư'] = 'Chung cư mini'
    elif 'Nhà tập thể' in row['Dự án'] or 'Nhà ở xã hội' in row['Dự án']:
        dtset.at[index, 'Loại hình chung cư'] = 'Nhà tập thể'
    else:
        dtset.at[index, 'Loại hình chung cư'] = 'Chung cư'

In [29]:
dtset.head(3)

,Tiêu đề,Dự án,Khu vực,Đường/Phố,Phường/Xã,Quận/Huyện,Phân loại quận,Loại tin,Mã tin,Thông tin mô tả,Diện tích,Mức giá,Hướng nhà,Hướng ban công,Số phòng ngủ,Số toilet,Pháp lý,Nội thất,Loại hình chung cư
0,Chính chủ bán căn hoa hậu view thoáng tầng tru...,Golden Palace,"đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.",Mễ Trì,Mễ Trì,Nam Từ Liêm,Nội thành,Tin thường,37587338,Chính chủ bán căn hoa hậu view thoáng tầng tru...,128.0,35.0,Tây - Bắc,Đông - Nam,3.0,2.0,NaN,NaN,Chung cư
1,"Bán căn hộ CT5 Văn Khê, Hà Đông",CT5 Văn Khê,"Phường La Khê, Hà Đông, Hà Nội",NaN,La Khê,Hà Đông,Nội thành,Tin thường,39014297,"Bán căn hộ CT5A, Văn Khê, Hà Đông.\n- Diện tíc...",82.0,32.3,Tây - Bắc,Đông - Nam,2.0,2.0,Sổ hồng,Đầy đủ,Chung cư
2,Cần tiền bán nhanh căn hộ cao cấp tòa Diamond ...,Diamond Flower Tower,"Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh X...",Hoàng Đạo Thúy,Nhân Chính,Thanh Xuân,Nội thành,Tin thường,39135117,- Chính chủ cần tiền đầu tư nhờ bán gấp căn hộ...,115.0,55.0,Nam,NaN,3.0,2.0,Sổ đỏ/ Sổ hồng,Đầy đủ,Chung cư


### Handle "Thông tin mô tả" column

In [30]:
# Initialize additional columns with default value 0
dtset['Nhóm 1-Trường học'] = 0
dtset['Nhóm 1-Chợ/Siêu thị'] = 0
dtset['Nhóm 1-Bệnh viện'] = 0
dtset['Nhóm 2-Công viên'] = 0
dtset['Nhóm 2-Bể bơi'] = 0
dtset['Nhóm 2-Gym'] = 0
dtset['Slot ô tô'] = 0
dtset['Nhà thoáng'] = 0
dtset['Hiện đại'] = 0
dtset['Bán gấp'] = 0
dtset['Lô góc'] = 0
dtset['Hỗ trợ vay'] = 0

# School data
school_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('trường|học viện', case=False, na=False)) &
    ~((dtset['Thông tin mô tả'].str.contains('thị trường|môi trường|quảng trường|Trường Chinh|mội trường|chuyển trường|Mr Trường|Mr. Trường|em Trường|trường hợp|Chú Trường|chuyểntrường|tiếp Trường|tim trường|Trường Sa|Trường Anh|Liên hệ Trường|công trường|Nguyễn Trường Tộ|ij|giáo viên trường|Xuân Trường', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('Tuệ|Vin|Kim Đồng|Lomo|chợ, trường|trường công|trường Nguyễn|Niutons|Ams|Unit|Brighton|trường trạm|Xanh|trường - chợ|Sư phạm|Đại Hoc|tiện trường|C1|học|Chu Văn An|mần non|trường chợ|top|đủ trường|trường Am|các cấp|trường chuyên|College|trường Đông Thái|học viện|trường quốc tế|trường, chợ|cao đẳng|gần trường|trường TH|mẫu giáo|nhà trẻ|giáo dục|tiểu học|mầm non|cấp 1|cấp 2|cấp 3|trường bưu chính|liên cấp|trường học|ĐH|đại học', case=False, na=False)))]
dtset.loc[school_data.index, 'Nhóm 1-Trường học'] = 1

# Market data
market_data = dtset[dtset['Thông tin mô tả'].str.contains('chợ|siêu thị|trung tâm thương mại', case=False, na=False)]
dtset.loc[market_data.index, 'Nhóm 1-Chợ/Siêu thị'] = 1

# Hospital data
hospital_data = dtset[dtset['Thông tin mô tả'].str.contains('bệnh viện|trung tâm chăm sóc|Vinmec', case=False, na=False)]
dtset.loc[hospital_data.index, 'Nhóm 1-Bệnh viện'] = 1

# Play area data
play_data = dtset[dtset['Thông tin mô tả'].str.contains('khu vui chơi|công viên|trẻ em|vườn hoa', case=False, na=False)]
dtset.loc[play_data.index, 'Nhóm 2-Công viên'] = 1

# Sport data
sport_data = dtset[dtset['Thông tin mô tả'].str.contains('bể bơi', case=False, na=False)]
dtset.loc[sport_data.index, 'Nhóm 2-Bể bơi'] = 1

# Gym data
gym_data = dtset[dtset['Thông tin mô tả'].str.contains('gym', case=False, na=False)]
dtset.loc[gym_data.index, 'Nhóm 2-Gym'] = 1

# Slot data
slot_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('ô tô|hầm|slot', case=False, na=False)) &
    (dtset['Thông tin mô tả'].str.contains('ô tô|hầm|slot xe|slot oto|slot đỗ|slot để|ôtô|sloto|otô|có slot|oto|sổ sẵn slot', case=False, na=False))
]
dtset.loc[slot_data.index, 'Slot ô tô'] = 1

# Floor data
low_data = dtset[dtset['Thông tin mô tả'].str.contains('tầng thấp', case=False, na=False)]
mid_data = dtset[dtset['Thông tin mô tả'].str.contains('tầng trung', case=False, na=False)]
high_data = dtset[dtset['Thông tin mô tả'].str.contains('tầng cao', case=False, na=False)]
dtset['Tầng'] = None
dtset.loc[low_data.index, 'Tầng'] = 'Thấp'
dtset.loc[mid_data.index, 'Tầng'] = 'Trung'
dtset.loc[high_data.index, 'Tầng'] = 'Cao'

# Airy data
thoang_data = dtset[dtset['Thông tin mô tả'].str.contains('view|thoáng', case=False, na=False)]
dtset.loc[thoang_data.index, 'Nhà thoáng'] = 1

# Modern data
modern_data = dtset[dtset['Thông tin mô tả'].str.contains('hiện đại|sang trọng|xịn xò|xịn sò|sịn xò|cao cấp', case=False, na=False)]
dtset.loc[modern_data.index, 'Hiện đại'] = 1

# Urgent sale data
bangap_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('gấp|cắt lỗ|bán nhanh', case=False, na=False)) &
    ~((dtset['Thông tin mô tả'].str.contains('gấp đôi|Gọi gấp|Xem nhà gấp|gấp 2|gấp bội|đắt gấp|gấp rút hoàn thành|Liên hệ gấp|gấp rút|Lh gấp|Em Trang Gấp|tăng giá|LH|mua gấp|tăng gấp|xem gấp|Gấp.', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('vốn xoay|giao dịch|bay gấp|biệt thự|Gấp hạ|Gấp,|chủ gấp|bán nhanh|sang gấp|gấp bán|cắt lỗ|thu tiền|gấp trong|xử lý|gấp anh|bán nhà gấp|Gấp -|cắt lỗ gấp|ban gấp|gấp nhà|chốt gấp|bán ngay|việc gấp|muốn gấp|nhượng lại|Gấp gấp|bấn gấp|bân gấp|cần gấp|dòng tiền|gấp chỉ|cần tiền gấp|cực gấp|bán lại|lo việc gấp|thanh khoản gấp|Giá đẩy gấp|rất gấp|nhượng gấp|thanh lý gấp|ra gấp|chuyển đổi gấp|gấp lắm|bán gấp', case=False, na=False)))
]
dtset.loc[bangap_data.index, 'Bán gấp'] = 1

# Corner unit data
goc_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('căn góc|căn hộ góc', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('căn hộ cạnh căn góc', case=False, na=False))
]
dtset.loc[goc_data.index, 'Lô góc'] = 1

# Loan support data
filtered_data = dtset[
    (dtset['Thông tin mô tả'].str.contains('hỗ trợ', case=False, na=False)) &
    ~(dtset['Thông tin mô tả'].str.contains('không hỗ trợ', case=False, na=False))
]
dtset.loc[filtered_data.index, 'Hỗ trợ vay'] = 1

In [31]:
dtset.head(3)

,Tiêu đề,Dự án,Khu vực,Đường/Phố,Phường/Xã,Quận/Huyện,Phân loại quận,Loại tin,Mã tin,Thông tin mô tả,...,Nhóm 2-Công viên,Nhóm 2-Bể bơi,Nhóm 2-Gym,Slot ô tô,Nhà thoáng,Hiện đại,Bán gấp,Lô góc,Hỗ trợ vay,Tầng
0,Chính chủ bán căn hoa hậu view thoáng tầng tru...,Golden Palace,"đường Mễ Trì, Phường Mễ Trì, Nam Từ Liêm, Hà Nội.",Mễ Trì,Mễ Trì,Nam Từ Liêm,Nội thành,Tin thường,37587338,Chính chủ bán căn hoa hậu view thoáng tầng tru...,...,0,0,0,1,1,0,0,0,1,Trung
1,"Bán căn hộ CT5 Văn Khê, Hà Đông",CT5 Văn Khê,"Phường La Khê, Hà Đông, Hà Nội",NaN,La Khê,Hà Đông,Nội thành,Tin thường,39014297,"Bán căn hộ CT5A, Văn Khê, Hà Đông.\n- Diện tíc...",...,0,0,0,0,0,0,0,0,0,None
2,Cần tiền bán nhanh căn hộ cao cấp tòa Diamond ...,Diamond Flower Tower,"Phố Hoàng Đạo Thúy, Phường Nhân Chính, Thanh X...",Hoàng Đạo Thúy,Nhân Chính,Thanh Xuân,Nội thành,Tin thường,39135117,- Chính chủ cần tiền đầu tư nhờ bán gấp căn hộ...,...,0,0,0,1,1,1,1,0,0,None


### Drop unvalued column

In [32]:
# Print the initial dataset size
print("Initial dataset size:", dtset.shape)

# Drop specified columns
dtset.drop(['Khu vực', 'Tiêu đề', 'Thông tin mô tả'], axis=1, inplace=True)

# Print the dataset size after removing some attribute columns
print("Dataset size after removing some attribute columns:", dtset.shape)

Initial dataset size: (14953, 32)
Dataset size after removing some attribute columns: (14953, 29)


In [33]:
# Update the list of columns after removing some columns
dtset_columns_after = dtset.columns.tolist()

In [34]:
# Summary
# Initialize an empty list to store removed columns
delete_col_count = []

# Iterate through the original list of columns
for element in dtset_columns:
    # Check if the column is not in the updated list of columns
    if element not in dtset_columns_after:
        # If the column is not in the updated list, append it to the list of removed columns
        delete_col_count.append(element)

# Print the removed columns and their count
print("The removed attribute columns:", delete_col_count)
print("Total number of removed columns:", len(delete_col_count))

The removed attribute columns: ['Tiêu đề', 'Khu vực', 'Ngày đăng', 'Ngày hết hạn', 'Người đăng', 'Thông tin mô tả']
Total number of removed columns: 6


In [35]:
dtset.shape

(14953, 29)

In [36]:
# Save the DataFrame to a CSV file 
dtset.to_csv('chungcu_eng.csv', index=False)

In [37]:
# Save the "Thỏa thuận" DataFrame to a CSV file 
new_file = dtset[dtset['Mức giá'] == 'Thỏa thuận']
new_file.to_csv('thoa_thuan_chungcu.csv', index=False)

In [38]:
# Save the remaining DataFrame to a CSV file 
remaining_file = dtset[dtset['Mức giá'] != 'Thỏa thuận']
remaining_file.to_csv('remaining_chungcu.csv', index=False)